In [1]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import os
import numpy as np

#exception 처리 대비
from selenium.common.exceptions import UnexpectedAlertPresentException, NoSuchElementException

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install finance-datareader
import FinanceDataReader as fdr
stocks = fdr.StockListing('KRX')
stocks = stocks.drop(['Sector', 'Industry', 'ListingDate', 'SettleMonth', 'Representative', 'HomePage', 'Region'], axis = 1)

kosdaq = stocks[stocks['Market'] == 'KOSDAQ']

mask = kosdaq['Symbol'].str.len() == 6
kosdaq = kosdaq.loc[mask]
kosdaq = kosdaq.reset_index(drop = True, inplace = False)

import re
def search_words(text):
  result = re.findall(r'\d+', text)
  return " ".join(result)

kosdaq['extract'] = kosdaq['Symbol'].apply(lambda x: search_words(x))

mask = kosdaq['extract'].str.len() == 6
kosdaq = kosdaq.loc[mask]
kosdaq = kosdaq.reset_index(drop = True, inplace = False)

     |████████████████████████████████| 127 kB 7.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.0.0 requires urllib3[secure]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') 
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# 아래 셀에서 Mozilla ~ 로 시작하는 부분을 자신의 컴퓨터에 맞게 바꿔주어야 함.
# http://www.useragentstring.com/
# 위 링크 들어가서 확인하기
chrome_options.add_argument(f'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.implicitly_wait(10)

title_news = []
date_news = []
content_news = []
company = []

url_base1 = 'https://finance.naver.com/item/news_news.naver?code='
url_base2 = '&page='
url_base3 = '&sm=title_entity_id.basic&clusterId='

# 여기서 자신이 맡은 숫자로 바꿔주기
## EX) 500부터 1000까지라면 range(500, 1000, 1)
###########################################
# 숫자 그대로 넣으세요!! 다 고려해서 배정한 거임!!! #
#          괜히 1 더해서 넣지 마 !!!!!!      #
#########################################
for k in range(289, 300, 1):
  print('-'*30)
  print(f'{k}번째 입니다.')
  print(f"\n {kosdaq['Name'][k]}")
  for i in range(1, 200, 1):
    url = url_base1 + kosdaq['extract'][k] + url_base2 + str(i) + url_base3
    driver.get(url)
    time.sleep(2)
    try:
      num = 0
      dates = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[1]/td[3]').text
    except:
      num = 99
    
    if num == 0:
      dates = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[1]/td[3]').text
      date = pd.to_datetime(dates)
      if (date >= pd.to_datetime('2021-10-01')):
        for j in range(1, 21, 1):
          try:
            url = url_base1 + kosdaq['extract'][k] + url_base2 + str(i) + url_base3
            driver.get(url)
            num = 1
            elem = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td[1]/a')
          except:
            url = url_base1 + kosdaq['extract'][k] + url_base2 + str(i) + url_base3
            driver.get(url)
            try:
              num = 2
              elem = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td/table/tbody/tr[1]/td[1]/a')
            except:
              num = 3
          if num == 1:
            companies = kosdaq['Name'][k]
            company.append(companies)
            driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td[1]/a').click()
            time.sleep(2)
            titles = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table/tbody/tr[1]/th/strong').text
            print('\n제목:')
            print(titles)
            title_news.append(titles)
            dates = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table/tbody/tr[2]/th/span/span').text
            date = pd.to_datetime(dates)
            print('\n날짜')
            print(date)
            date_news.append(date)
            contents = driver.find_element_by_id('news_read').text
            print('\n본문')
            print(contents)
            content_news.append(contents)
          elif num == 2:
            companies = kosdaq['Name'][k]
            company.append(companies)
            titles = driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td/table/tbody/tr[1]/td[1]/a').text
            print('\n제목:')
            print(titles)
            title_news.append(titles)
            driver.find_element_by_xpath(f'/html/body/div/table[1]/tbody/tr[{j}]/td/table/tbody/tr[1]/td[1]/a').click()
            time.sleep(2)
            dates = driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/table/tbody/tr[2]/th/span/span').text
            date = pd.to_datetime(dates)
            print('\n날짜')
            print(date)
            date_news.append(date)
            contents = driver.find_element_by_id('news_read').text
            print('\n본문')
            print(contents)
            content_news.append(contents)
          elif num == 3:
            break
      else:
        break
    elif num == 99:
      break
            



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


------------------------------
289번째 입니다.

 디아이티


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dis


제목:
[코스닥 공시] 포인트엔지니어링 / 디아이티 / 알톤스포츠

날짜
2021-10-20 17:35:00

본문
◇포인트엔지니어링=20억원 자기주식취득 신탁 계약 체결.

◇디아이티=삼성디스플레이와 119억원 제조 장비 계약 체결.

◇알톤스포츠=카카오모빌리티와 92억원 전기자전거 공급계약 체결.

제목:
디아이티, 119억원 규모 디스플레이 제조장비 공급

날짜
2021-10-20 14:46:00

본문
[이데일리 유준하 기자] 디아이티(110990)는 삼성디스플레이에 119억1000만원 규모 디스플레이 제조 장비를 공급했다고 20일 공시했다.

이는 지난해 매출액 대비 41.64% 규모이며 계약기간은 오는 2022년 5월31일까지다.

제목:
[코스닥 공시] 하나기술 / 게임빌 / 디아이티

날짜
2021-09-29 17:34:00

본문
◇하나기술=2차전지 시스템 특허권 취득.

◇게임빌=종속회사 게임빌플러스의 코인원 지분 15만1218주 540억원 규모로 추가 취득.

◇디아이티=삼성디스플레이와 68억원 규모 디스플레이 제조장비 공급계약.

제목:
디아이티, 삼성디스플레이와 69억원 규모 디스플레이 제조장비 공급계약

날짜
2021-09-29 16:08:00

본문
[이데일리 권효중 기자] 디아이티(110990)는 삼성디스플레이와 약 69억원 규모의 디스플레이 제조장비 공급 계약을 체결했다고 29일 공시했다.

이는 지난해 매출액 대비 24.03%에 해당하며, 계약기간은 지난 28일부터 오는 2022년 4월 30일까지다.

제목:
디아이티, 삼성디스플레이와 69억 규모 공급계약 체결

날짜
2021-09-29 15:43:00

본문
[헤럴드경제=증권부] 디아이티는 삼성디스플레이와 디스플레이 제조장비 공급계약을 체결했다고 29일 공시했다.

계약금액은 68억7000만원으로 이는 2020년 매출 대비 24.03%에 해당하는 규모이다.

계약기간은 2022년 4월 30일까지다.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


------------------------------
290번째 입니다.

 디알젬

제목:
디알젬, 20억원 규모 자기주식 취득

날짜
2021-11-02 17:16:00

본문
[이데일리 유준하 기자] 디알젬(263690)은 주가 안정을 통한 주주이익 제고를 위해 20억원 규모 자기주식 취득을 결정했다고 2일 공시했다. 취득 예상기간은 오는 3일부터 2022년 2월2일까지다.

제목:
EU와 2억회분 계약에 '노바백스 관련주' 동반상승…SK바사·디알젬·켐온↑

날짜
2021-08-05 11:30:00

본문
노바백스가 EU에 2억회분의 백신 공급 계약을 체결하면서 관련주가 강세다. <로이터연합 제공>
미국 제약기업 노바백스가 유럽연합(EU)와 백신 공급계약을 체결했다는 소식에 SK바이오사이언스·디알젬·켐온 등 관련주가 강세다.

한국거래소에 따르면 8월 5일 오전 10시30분 기준 SK바이오사이언스는 전일 대비 4.51% 오른 22만원에 거래되고 있다. SK바이오사이언스는 지난해 8월 노바백스와 백신 위탁생산 계약을 체결해 현재 노바백스 백신 시생산 중에 있다.

이외 켐온(3.28%)과 디알젬(2.67%)도 노바백스 관련주로 거론되며 상승하고 있다. 켐온은 앞서 SK바이오사이언스의 백신 비임상 연구를 위탁 수행한 바 있고, 디알젬은 2대주주인 후지필름의 자회사가 노바백스 백신 후보물질 위탁제조를 담당한 바 있다.

앞서 지난 4일(현지시간) 노바백스는 유럽 집행위원회 측과 자사의 코로나19 백신 후보인 ‘NVX-CoV2373' 선구매 계약에 최종 합의했다고 밝혔다. 올해 4분기부터 최대 1억회분의 백신을 EU에 제공하고, 유럽 의약품 규제당국(EMA)의 승인 후 유럽이 선택권을 행사하면 2023년까지 추가로 1억회분을 더 제공하게 된다는 설명이다.

이번에 공급계약이 체결된 노바백스 백신 ‘NVX-CoV2373'은 백신의 면역반응 강화를 돕기 위한 항원보강제 메이트릭스(Matrix)-M이 첨가된 코로나19 백신 후보물질이다. 앞서 미국과 멕시코에서 약 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead



제목:
디알젬, 주당 200원 결산배당


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead



날짜
2021-02-01 14:30:00

본문
[이데일리 박정수 기자] 디알젬(263690)은 보통주 1주당 200원의 현금 결산배당을 결정했다고 1일 공시했다. 시가배당율은 1.2%이며 배당금 총액은 22억3840만원이다. 배당기준일은 작년 12월 31일이다.

회사 측은 “배당금지급 예정일은 정기주주총회일(3월 29일)로부터 1개월 이내에 지급할 예정”이라고 전했다.

박정수 (ppjs@edaily.co.kr)

▶ #24시간 빠른 #미리보는 뉴스 #eNews+
▶ 네이버에서 '이데일리 뉴스'를 만나보세요
▶ 빡침해소, 청춘뉘우스 '스냅타임'


＜ⓒ종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지＞

제목:
디알젬, 지난해 영업익 233억…전년비 409%↑

날짜
2021-02-01 14:28:00

본문
[이데일리 박정수 기자] 디알젬(263690)은 지난해 영업이익 233억767만원으로 전년 대비 409.7% 증가한 것으로 잠정 집계됐다고 1일 공시했다. 같은 기간 매출액은 1066억370만원으로 77.4% 증가했고, 순이익은 202억9150만원으로 270.9% 늘었다.

박정수 (ppjs@edaily.co.kr)

▶ #24시간 빠른 #미리보는 뉴스 #eNews+
▶ 네이버에서 '이데일리 뉴스'를 만나보세요
▶ 빡침해소, 청춘뉘우스 '스냅타임'


＜ⓒ종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지＞

제목:
디알젬, 지난해 영업익 233억…전년비 409.7%↑

날짜
2021-02-01 11:36:00

본문
[헤럴드경제=증권부] 디알젬은 지난해 연결기준 영업이익이 233억767만원으로 전년대비 409.7% 증가했다고 1일 공시했다.

같은 기간 매출액은 1066억371만원으로 77.4% 늘어난 것으로 나타났다.

당기순이익은 202억9151만원으로 270.9% 증가한 것으로 집계됐다.

totoro@heraldcorp.com

▶환경적 대화기구 '헤럴드에코'
▶밀리터리 전문 콘텐츠 ‘헤밀’
▶헤럴드경제 네이버 채널 구독



In [6]:
df = {'company': company, 'title': title_news, 'content': content_news, 'date' : date_news}
df = pd.DataFrame(df)

In [14]:
# 파일명만 코스닥(자신이 맡은 범위)로 써주세요
## EX) 코스닥(500~1000).csv
df.to_csv('/content/drive/Shareddrives/🥑PSAT_DM_2728🥑/Dataset/[종목크롤링]/코스닥/코스닥(0~300).csv', index = False, encoding = 'utf-8')